In [8]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import  pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

c:\Users\Dawood Khan\anaconda3\envs\llm\lib\site-packages\ibm_watson_machine_learning\foundation_models\extensions\langchain\llm.py:60: WatsonxLLMDeprecationWarning: ibm_watson_machine_learning.foundation_models.extensions.langchain.WatsonxLLM is deprecated and will not be supported in the future. Please import from langchain-ibm instead.
To install langchain-ibm run `pip install -U langchain-ibm`.
  _raise_watsonxllm_deprecation_warning()


In [11]:
filename = 'CompanyPolicies.txt'
with open(filename, 'r') as file:
    contents= file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

In [12]:
loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(documents)
print(len(texts))

Created a chunk of size 1624, which is longer than the specified 1000
Created a chunk of size 1885, which is longer than the specified 1000
Created a chunk of size 1903, which is longer than the specified 1000
Created a chunk of size 1729, which is longer than the specified 1000
Created a chunk of size 1678, which is longer than the specified 1000
Created a chunk of size 2032, which is longer than the specified 1000
Created a chunk of size 1894, which is longer than the specified 1000


16


In [13]:
embeddings= HuggingFaceEmbeddings()
doc_search = Chroma.from_documents(texts, embeddings) # Store the embeddings in doc_search using Chroma
print('document ingested')

C:\Users\Dawood Khan\AppData\Local\Temp\ipykernel_2096\1299915543.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings()
C:\Users\Dawood Khan\AppData\Local\Temp\ipykernel_2096\1299915543.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings= HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Dawood Khan\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dawood Khan\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

document ingested


In [18]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

model_id = "google/gemma-2-2b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.5,
    device=0  # or -1 for CPU
)

Device set to use cuda:0


In [23]:
model = HuggingFacePipeline(pipeline = pipe)

In [24]:
qa = RetrievalQA.from_chain_type(llm = model,
                                 chain_type = 'stuff',
                                 retriever = doc_search.as_retriever(),
                                 return_source_documents = False)

query = 'What is the Mobile Policy?'
qa.invoke(query)

{'query': 'What is the Mobile Policy?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n4.\tMobile Phone Policy\n\nThe Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance.\nAcceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations.\nSecurity: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device.\nConfidentiality: Avoid transmitting sensitive company informat

In [ ]:
# Another Highlevel Question

qa = RetrievalQA.from_chain_type(llm = model,
                                 chain_type = 'stuff',
                                 retriever = doc_search.as_retriever(),
                                 return_source_documents = True)
query = 'Can you summarize the document for me?' 
qa.invoke(query)

In [31]:
# another question
qa = RetrievalQA.from_chain_type(llm = model,
                                 chain_type ='stuff',
                                 retriever = doc_search.as_retriever(),
                                 return_source_documents = False)
query = 'Can I eat in the Company vehicles?' 
qa.invoke(query)

KeyboardInterrupt: 

# Using Prompt Template

In [33]:
prompt_template = """Use the information formt he document to answer the question at the end. If you Donot Know the answer, then just say that you don't know the answer and donot make any answers by yourself.
\n{context}\n
Question: {question}"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables=['context', 'question']

)

chain_type_kwargs = {'prompt':PROMPT}

In [34]:
qa = RetrievalQA.from_chain_type(llm = model,
                                 chain_type = 'stuff',
                                 retriever = doc_search.as_retriever(),
                                 chain_type_kwargs = chain_type_kwargs, 
                                 return_source_documents = False)

query = 'Can I eat in Company Vehicles?'
qa.invoke(query)

{'query': 'Can I eat in Company Vehicles?',
 'result': "Use the information formt he document to answer the question at the end. If you Donot Know the answer, then just say that you don't know the answer and donot make any answers by yourself.\n\nPolicy Purpose: The Smoking Policy has been established to provide clear guidance and expectations concerning smoking on company premises. This policy is in place to ensure a safe and healthy environment for all employees, visitors, and the general public.\nDesignated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to maintain the overall cleanliness of the premises.\nSmoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.\nCompliance with Applicable Laws: All employees and visitors must 

# Make the Conversation have memory

In [35]:
memory = ConversationBufferMemory(memory_key = 'chat_history',
                                  return_messages=True)

C:\Users\Dawood Khan\AppData\Local\Temp\ipykernel_2096\2514505429.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = 'chat_history',


In [ ]:
# Creating a Conversational Retrieval Chain
qa = ConversationalRetrievalChain.from_llm(llm = model,
                                           chain_type = 'stuff',
                                           retriever = doc_search.as_retriever(),
                                           memory = memory,
                                           get_chat_history = lambda h:h,
                                           return_source_documents = False)



TypeError: ConversationalRetrievalChain.from_llm() missing 1 required positional argument: 'llm'

In [ ]:
history = []

In [ ]:
query = 'What is mobile Policy?'
result = qa.invoke({'question':query},{'chat_history':history})
print(result['answer'])

In [ ]:
# append the previous queries and results to the history 
history.append((query, result['answer']))


In [ ]:
query = 'List points in it?'
result = qa(({'question':query},{'chat_history':history}))
print(result['answer'])

In [ ]:
# Append to histroy again
history.append((query, result['answer']))

In [ ]:
query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

# Make an Agent of the RAG 

In [37]:
def qa():
    memory = ConversationBufferMemory(memory_key = 'chat_history', return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm = model,
                                               retriever = doc_search.as_retriever(),
                                               memory = memory,
                                               get_chat_history = lambda h:h,
                                               return_source_documents = False)
    
    history=[]

    while True:
        query  = input('Question: ')

        if query.lower() in ['quit','exit','bye']:
            print('Answer: Goodbye!')
            break

        result = qa({'question':query}, {'chat_history':history})

        history.append((query, result['answer']))

        print('Answer: ', result['answer'])

In [ ]:
qa()

C:\Users\Dawood Khan\AppData\Local\Temp\ipykernel_2096\1660666326.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'question':query}, {'chat_history':history})


Answer:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

4.	Mobile Phone Policy

The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance.
Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations.
Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device.
Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be disc